# pCrunch's Crunch class

The Crunch class is a general analysis tool for batches of time-series based data across multiple environmental conditions (i.e., a full wind speed and turbulence seed sweep). The methods are agnostic to the aeroelastic multibody simulation tool (OpenFAST or HAWC2 or Bladed or QBlade or in-house equivalents). The AeroelasticOutput class provides the data containers for each individual simulation.  The AeroelasticOutput class provides many analysis capabilities and the Crunch class extends them into their batch versions.

The Crunch class supports keeping all time series data in memory and a lean "streaming" version where outputs are processed and then deleted, retaining only the critical statistics and analysis outputs.

This file lays out some workflows and showcases capabilities of the Crunch class.  It is probably best to walk through the examples of the AeroelasticOutput class first.

## Creating a new class instance

The `Crunch` class can be initialized from a list of AeroelasticOutput instances or none, in order to setup a "streaming" analysis.  Pleaes see the AeroelasticOutput example for the various means to initialize one of its instances.  pCrunch provides a reader for OpenFAST output files (both binary and ascii) and common Python data structures are also supported.  To extend pCrunch for use with other aeroelastic multibody codes, users could simply use the `openfast_readers.py` file as a template.  Here are some examples:

In [21]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pCrunch import Crunch, read, FatigueParams

thisdir = os.path.realpath('')
datadir = os.path.join(thisdir, '..', 'pCrunch', 'test', 'data')

# OpenFAST output files
filelist = glob.glob( os.path.join(datadir, '*.out') )
filelist.sort()
print(f"Found {len(filelist)} files.")

# Read all outputs into a list
outputs = [read(m) for m in filelist[1:]]

# Vector magnitudes
mc = {
    "RootMc1": ["RootMxc1", "RootMyc1", "RootMzc1"],
    "RootMc2": ["RootMxc2", "RootMyc2", "RootMzc2"],
    "RootMc3": ["RootMxc3", "RootMyc3", "RootMzc3"],
}

# Channel-specific fatigue properties
fc = {
    "RootMc1": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
    "RootMc2": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
    "RootMc3": FatigueParams(lifetime=25.0, slope=10.0, ultimate_stress=6e8, load2stress=250.0, S_intercept=5e9),
}

# Channels to focus on for extreme event tabulation
ec = [
    "RotSpeed",
    "RotThrust",
    "RotTorq",
]

# Standard use case with all outputs read prior to use of Crunch.
mycruncher = Crunch(outputs)

# Can also add some batch data operations in the constructor (many more available in Batch Processing below)
mycruncher_mc = Crunch(outputs, magnitude_channels=mc, trim_data=[40, 80], fatigue_channels=fc, extreme_channels=ec)

# When planning on adding outputs later, you still need create a Crunch object that is initially empty of data
# The `lean` flag says that the outputs should be processed, but not stored in memory
mycruncher_lean = Crunch(outputs=[], lean=True)

# Can still add the batch operations to be done later when outputs are added
mycruncher_lean_mc = Crunch(outputs=[], lean=True, magnitude_channels=mc, trim_data=[40,80], fatigue_channels=fc, extreme_channels=ec)

Found 4 files.


## Crunching the data

### With full memory storage
The Crunch class can batch process the outputs using one or more processors up to the number available workstation cores.  This computes the essential statistics for each output.

In [22]:
# Process all outputs in parallel
mycruncher.process_outputs(cores=1)

# Process all outputs and override any prior input setting (especially in fatigue calculation)
mycruncher_mc.process_outputs(return_damage=True)

The key outputs that are stacked together for each output are:

- Summary statistics
- Extreme event table
- Damage equivalent loads (DELs)
- Palmgren-Miner damage

In [23]:
# The summary stats per each file are here:
mycruncher.summary_stats

,Time,WindVxi,WindVyi,WindVzi,WaveElev,Wave1Vxi,Wave1Vyi,Wave1Vzi,Wave1Axi,Wave1Ayi,...,Fair8Ang,Anch8Ten,Anch8Ang,TipSpdRat,RotCp,RotCt,RotCq,RootMc1,RootMc2,RootMc3
,min,min,min,min,min,min,min,min,min,min,...,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated,integrated
DLC2.3_1.out,40.0,8.200,0.0,0.0,-0.812,-0.5420,0.0,-0.7480,-0.7230,0.0,...,3289.97500,0.0,0.0,200.575785,7.928011,19.910354,1.038984,205721.341721,213361.952123,222926.620682
DLC2.3_2.out,40.0,8.197,0.0,0.0,-1.055,-0.5438,0.0,-0.5059,-0.3195,0.0,...,3284.85275,0.0,0.0,202.995350,8.196020,20.209087,1.063160,209783.673742,218196.946790,226368.344817
DLC2.3_3.out,40.0,8.197,0.0,0.0,-1.135,-0.5403,0.0,-0.5156,-0.2482,0.0,...,3286.98625,0.0,0.0,202.480952,8.109685,20.148358,1.055470,209258.712717,216929.128758,225974.727664


In [24]:
# These are indexable by channel, stat:
mycruncher.summary_stats["RootMc1"]

,min,max,std,mean,median,abs,integrated
DLC2.3_1.out,459.805830,9134.167593,2707.224813,5142.646328,6147.974788,9134.167593,205721.341721
DLC2.3_2.out,277.648587,9079.452302,2709.503118,5244.431821,6430.205456,9079.452302,209783.673742
DLC2.3_3.out,347.604352,8986.223847,2707.345218,5231.190499,6669.062227,8986.223847,209258.712717


In [25]:
mycruncher.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [26]:
# Or by file
mycruncher.summary_stats.loc["DLC2.3_1.out"]

Time      min               40.000000
WindVxi   min                8.200000
WindVyi   min                0.000000
WindVzi   min                0.000000
WaveElev  min               -0.812000
                            ...      
RotCt     integrated        19.910354
RotCq     integrated         1.038984
RootMc1   integrated    205721.341721
RootMc2   integrated    213361.952123
RootMc3   integrated    222926.620682
Name: DLC2.3_1.out, Length: 952, dtype: float64

In [27]:
# Damage equivalent loads are found here:
mycruncher_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,2557.248362,5801.906159,2074.270412
DLC2.3_2.out,2759.655817,4632.573610,2138.262799
DLC2.3_3.out,2791.460474,5839.945621,2213.542813


In [28]:
# Palmgren-Miner damage can be viewed with:
mycruncher_mc.damage

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,9.208221e-31,3.327773e-27,1.146698e-31
DLC2.3_2.out,1.972404e-30,3.504854e-28,1.537632e-31
DLC2.3_3.out,2.211880e-30,3.552505e-27,2.174332e-31


In [29]:
# Extreme events table. For each channel, there is a list of the extreme condition for each output case
mycruncher_mc.extremes

{'RotSpeed': [{'Time': 61.8,
   'RotSpeed': 11.1,
   'RotThrust': 369.0,
   'RotTorq': 844.0},
  {'Time': 61.9, 'RotSpeed': 11.28, 'RotThrust': 367.0, 'RotTorq': 159.4},
  {'Time': 61.9, 'RotSpeed': 11.33, 'RotThrust': 317.0, 'RotTorq': 140.8}],
 'RotThrust': [{'Time': 51.6,
   'RotSpeed': 10.1,
   'RotThrust': 759.0,
   'RotTorq': 2410.0},
  {'Time': 60.35, 'RotSpeed': 10.41, 'RotThrust': 786.9, 'RotTorq': -1041.0},
  {'Time': 60.35, 'RotSpeed': 10.48, 'RotThrust': 746.2, 'RotTorq': -1046.0}],
 'RotTorq': [{'Time': 54.45,
   'RotSpeed': 10.6,
   'RotThrust': 546.0,
   'RotTorq': 2650.0},
  {'Time': 54.4, 'RotSpeed': 10.74, 'RotThrust': 554.0, 'RotTorq': 2701.0},
  {'Time': 54.4, 'RotSpeed': 10.61, 'RotThrust': 575.1, 'RotTorq': 2638.0}]}

### Crunching in "lean / streaming" mode

If operating in "lean / streaming" mode, the outputs can either be processed one at a time, or even more lean, the summary statistics themselves can be passed to the `cruncher` object to append to the running list.

In [30]:
# Adding AeroelasticOutput objects in lean / streaming mode
for iout in outputs:
    mycruncher_lean.add_output( iout ) # Each output is processed without retaining the full time series

# Adding statistics incrementally, which is especially helpful when using parallel processing
# and finaly assembly of the full pool of outputs can still strain memory resources
results_pool = []
for iout in outputs:
    iresults = mycruncher_lean_mc.process_single( iout ) # This could be the result of parallelized function
    results_pool.append( iresults )

# After parallel processing is complete, assemble all the statistic for batch analysis
for iresults in results_pool:
    fname, stats, extremes, dels, damage =  iresults
    mycruncher_lean_mc.add_output_stats(fname, stats, extremes, dels, damage)

In [31]:
# Results are the same as the full-memory approach above
mycruncher_lean.summary_stats["RootMc1"]['min']

DLC2.3_1.out    459.805830
DLC2.3_2.out    277.648587
DLC2.3_3.out    347.604352
Name: min, dtype: float64

In [32]:
mycruncher_lean_mc.dels

,RootMc1,RootMc2,RootMc3
DLC2.3_1.out,2557.248362,5801.906159,2074.270412
DLC2.3_2.out,2759.655817,4632.573610,2138.262799
DLC2.3_3.out,2791.460474,5839.945621,2213.542813


## Integrating outputs with a probability weighting (AEP, Damage, etc)

When running design load cases, not all windspeeds, or other environmental condition, occur with equal likelihood.  pCrunch provides a way to assign a probability to each output.  This probability can then weight a summation to compute annual energy production (AEP), or sum all Palmgren-Miner damages together.  Using a subset of the outputs is also a provided capability.

pCrunch provides a couple different ways to set the probabilities, either:
- Inflow wind speed using a Weibull or Rayleigh distribution for the site
- IEC turbine class with different average wind speeds that define a Weibull distribution
- Users can set the probability values directly.

In [33]:
# Set probability based on wind speed channel name, Weibull distribution average of 7.5 m/s (shape factor input optional)
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='weibull', weibull_k=2.0)

# Or Rayleigh distribution using the same distribution average of 7.5 m/s
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='rayleigh')

# If you only want to use some of the outputs, but not all of them
mycruncher.set_probability_distribution('WindVxi', 7.5, kind='weibull', idx=[0,2])

# If you would rather specify the inflow wind speed directly to use in the probability distribution
mycruncher.set_probability_distribution([8,10,12], 7.5, kind='weibull')

# Can also set the probability based on IEC turbine class, again using a channel name of user input of wind speeds
mycruncher.set_probability_turbine_class('WindVxi', 2)
mycruncher.set_probability_turbine_class([8,10,12], 2)

# A savvy user can set the probability values directly (they will be rescaled to sum to one no matter what)
mycruncher.prob = np.array([0.1, 0.5, 0.4])

Once the probabilities are set, the user can use them to calculate AEP or total fatigue accumulation across the scenarios represented by each output.  For the AEP calculation, the user must specify the channel name.  Additional loss factors or restriction to certain indices are optional inputs.

In [34]:
# Probability weighted and unweighted AEP values are returned
mycruncher.compute_aep('GenPwr')

(33953594698.8, 33724052761.50001)

In [35]:
# Or with loss factors and restricted by select outputs
mycruncher.compute_aep('GenPwr', loss_factor=0.15, idx=[0,2])

(33707934777.6, 33486451832.250023)

In [36]:
# Damage calculation does not require a channel name, as it uses the previously computed case-specific and channel-specific values.
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue()

In [37]:
dels_tot

,RootMc1,RootMc2,RootMc3
Weighted,2702.788217,5424.808463,2142.025341
Unweighted,2702.788217,5424.808463,2142.025341


In [38]:
dams_tot

,RootMc1,RootMc2,RootMc3
Weighted,1.701702e-30,2.410254e-27,1.619554e-31
Unweighted,1.701702e-30,2.410254e-27,1.619554e-31


In [39]:
# Select indices are also available to restrict the summation
dels_tot, dams_tot = mycruncher_mc.compute_total_fatigue(idx=[0,2])

## Other Batch Procressing

The Crunch class provides batch extensions of nearly all of the operations offered in the AeroelasticOutputs class.  This includes the add channel or drop channel utilities and all statistical functions.  For the statistics, unlike the AeroelasticOutput class, these batch versions are functions, not data properties.  The result is returned as a list, with each index corresponding to the output list.  Many of these statistics also vary by channel, so there are likely to be nested lists.  Also, some are unavailable in "lean / streaming" mode.

In [40]:
# Adding channel
mycruncher.calculate_channel('LSSGagMya + LSSGagMza', 'Test')

# Adding Load Roses
lr = {'TwrBs': ['TwrBsFxt', 'TwrBsFyt']}
mycruncher.add_load_rose(lr, nsec=6)

# Dropping channels by string wildcard
mycruncher.drop_channel('Fair*')
mycruncher.drop_channel('Anch*')
mycruncher.drop_channel('Spn*')
mycruncher.drop_channel('Root*')
mycruncher.drop_channel('Wave*')
mycruncher.drop_channel('Ptfm*')
mycruncher.drop_channel('Tw*')
mycruncher.drop_channel('Yaw*')

Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300
Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300
Added channel, TwrBs0
Added channel, TwrBs60
Added channel, TwrBs120
Added channel, TwrBs180
Added channel, TwrBs240
Added channel, TwrBs300


In [41]:
# Indices to the minimum value for each channel
mycruncher.idxmins()

[array([  0, 448,   0,   0, 401, 401,   0,   0,   0,   0, 258, 798, 798,
        490, 262, 505, 284, 486, 247, 473, 788, 653, 552, 209, 553, 222,
          0, 522, 286, 137, 406, 507, 489, 798, 453, 555, 406, 492]),
 array([  0, 450,   0,   0, 401, 401,   0,   0,   0,   0, 373, 798, 798,
        490, 224, 507, 412, 487, 249, 474, 789, 656, 617, 720, 785,  18,
          0, 522, 288, 260, 406, 508, 489, 798, 453, 555, 406, 493]),
 array([  0, 450,   0,   0, 401, 401,   0,   0,   0,   0, 260, 798, 798,
        490, 431, 505, 412, 486, 249, 474, 788, 654, 617, 721, 553,  19,
          0, 522, 288, 260, 406, 506, 488, 798, 453, 555, 406, 492])]

In [42]:
# Indices to the maximum value for each channel
mycruncher.idxmaxs()

[array([800, 502,   0,   0, 313, 315,   0, 630, 630, 630, 487, 436, 438,
        364, 677, 257, 465, 234, 563, 284, 512, 555, 403, 298, 655, 551,
          0, 232, 109, 314, 289, 567, 515, 448, 292, 231, 289, 555]),
 array([800, 505,   0,   0, 307, 313,   0, 630, 630, 630, 260, 438, 438,
        363, 683, 259, 466, 235, 565, 285, 511, 783, 403, 552, 657, 618,
          0, 407, 114, 316, 288, 561, 515, 449, 291, 407, 289, 554]),
 array([800, 505,   0,   0, 307, 311,   0, 630, 630, 630, 486, 438, 438,
        600, 686, 262, 466, 238, 564, 288, 511, 782, 403, 552, 655, 617,
          0, 407, 113, 316, 288, 590, 515, 449, 291, 407, 289, 553])]

In [43]:
# Minimum value of each channel
mycruncher.minima()

[[40.0,
  8.2,
  0.0,
  0.0,
  -0.812,
  -0.542,
  0.0,
  -0.748,
  -0.723,
  0.0,
  -0.832,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0756,
  -0.032,
  -3.14,
  -0.0213,
  -0.307,
  -2.09,
  -0.938,
  0.0,
  -3.08,
  -0.894,
  0.0,
  -3.44,
  -1.22,
  0.0,
  9.61,
  9.39,
  9.52,
  -2.01,
  -0.132,
  -0.479,
  -0.82,
  -0.0485,
  0.0,
  19.5,
  -0.129,
  -1.01,
  -0.107,
  -5.44,
  -1.29,
  -0.351,
  -0.0192,
  -0.352,
  -127.0,
  -212.0,
  113.0,
  -3770.0,
  -5170.0,
  -80.0,
  -146.0,
  -205.0,
  -168.0,
  -4040.0,
  -6630.0,
  -67.5,
  -106.0,
  -208.0,
  -114.0,
  -3770.0,
  -5690.0,
  -66.7,
  -572.0,
  -1110.0,
  -54.6,
  -923.0,
  -1600.0,
  -50.1,
  -599.0,
  -1930.0,
  -48.3,
  -353.0,
  -1140.0,
  -1100.0,
  -1160.0,
  -3840.0,
  -3360.0,
  -1200.0,
  -65.3,
  -3620.0,
  -1310.0,
  -5210.0,
  -3360.0,
  -1680.0,
  -137.0,
  -7160.0,
  -4940.0,
  -135000.0,
  -3360.0,
  -2730.0,
  -67500.0,
  -3360.0,
  200.0,
  77.9,
  0.0,
  0.0,
  234.0,
  72.8,
  0.0,
  0.0,
  300.0,

In [44]:
# Maximum value of each channel
mycruncher.maxima()

[[80.0,
  12.7,
  0.0,
  0.0,
  1.11,
  0.875,
  0.0,
  0.67,
  0.68,
  0.0,
  0.537,
  2790.0,
  27.3,
  0.0,
  90.0,
  90.0,
  90.0,
  360.0,
  11.1,
  1090.0,
  0.00638,
  1.29,
  4.91,
  1.23,
  0.0,
  4.58,
  1.57,
  0.0,
  4.72,
  2.1,
  0.0,
  65.5,
  65.4,
  65.4,
  2.19,
  0.176,
  0.519,
  0.694,
  0.0272,
  0.0,
  37.2,
  0.0393,
  0.639,
  0.185,
  5.02,
  0.311,
  0.267,
  0.0103,
  0.383,
  250.0,
  170.0,
  675.0,
  4700.0,
  8870.0,
  58.2,
  232.0,
  184.0,
  637.0,
  4500.0,
  8190.0,
  59.5,
  238.0,
  156.0,
  623.0,
  4590.0,
  8550.0,
  66.7,
  1170.0,
  2390.0,
  12.5,
  1510.0,
  2250.0,
  18.4,
  808.0,
  2280.0,
  14.2,
  759.0,
  1090.0,
  1130.0,
  2650.0,
  2950.0,
  3590.0,
  1090.0,
  47.6,
  -3270.0,
  2690.0,
  2090.0,
  1010.0,
  1510.0,
  53.6,
  -6610.0,
  7220.0,
  117000.0,
  1000.0,
  4750.0,
  56800.0,
  1010.0,
  229.0,
  83.7,
  0.0,
  0.0,
  257.0,
  76.1,
  0.0,
  0.0,
  365.0,
  65.7,
  0.0,
  0.0,
  616.0,
  58.5,
  290.0,
  0.0,
  616.0,
 

In [45]:
# Maximum value of absolute values of each channel
mycruncher.absmaxima()

[array([8.00e+01, 1.27e+01, 0.00e+00, 0.00e+00, 2.79e+03, 2.73e+01,
        0.00e+00, 9.00e+01, 9.00e+01, 9.00e+01, 3.60e+02, 1.11e+01,
        1.09e+03, 2.13e-02, 1.29e+00, 4.91e+00, 1.23e+00, 4.58e+00,
        1.57e+00, 4.72e+00, 2.10e+00, 2.19e+00, 1.76e-01, 5.19e-01,
        8.20e-01, 4.85e-02, 0.00e+00, 7.59e+02, 1.14e+03, 1.13e+03,
        2.65e+03, 3.84e+03, 3.59e+03, 8.83e+00, 4.66e-01, 1.13e+00,
        6.24e-02, 4.93e+03]),
 array([8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00, 2.866e+03, 2.780e+01,
        0.000e+00, 9.000e+01, 9.000e+01, 9.000e+01, 3.595e+02, 1.128e+01,
        1.103e+03, 2.337e-02, 1.418e+00, 4.852e+00, 1.152e+00, 4.701e+00,
        1.658e+00, 4.526e+00, 2.183e+00, 2.472e+00, 1.646e-01, 5.903e-01,
        9.164e-01, 5.223e-02, 0.000e+00, 7.869e+02, 1.128e+03, 1.117e+03,
        2.701e+03, 3.969e+03, 3.930e+03, 8.988e+00, 4.801e-01, 1.171e+00,
        6.365e-02, 5.283e+03]),
 array([8.000e+01, 1.272e+01, 0.000e+00, 0.000e+00, 2.765e+03, 2.714e+01,
        0.00

In [46]:
# The range of data values (max - min)
mycruncher.ranges()

[[40.0,
  4.5,
  0.0,
  0.0,
  1.9220000000000002,
  1.417,
  0.0,
  1.4180000000000001,
  1.403,
  0.0,
  1.369,
  2790.0,
  27.3,
  0.0,
  90.0,
  90.0,
  90.0,
  359.9244,
  11.132,
  1093.14,
  0.02768,
  1.597,
  7.0,
  2.168,
  0.0,
  7.66,
  2.464,
  0.0,
  8.16,
  3.3200000000000003,
  0.0,
  55.89,
  56.010000000000005,
  55.88000000000001,
  4.199999999999999,
  0.308,
  0.998,
  1.5139999999999998,
  0.0757,
  0.0,
  17.700000000000003,
  0.1683,
  1.649,
  0.292,
  10.46,
  1.601,
  0.618,
  0.0295,
  0.735,
  377.0,
  382.0,
  562.0,
  8470.0,
  14040.0,
  138.2,
  378.0,
  389.0,
  805.0,
  8540.0,
  14820.0,
  127.0,
  344.0,
  364.0,
  737.0,
  8360.0,
  14240.0,
  133.4,
  1742.0,
  3500.0,
  67.1,
  2433.0,
  3850.0,
  68.5,
  1407.0,
  4210.0,
  62.5,
  1112.0,
  2230.0,
  2230.0,
  3810.0,
  6790.0,
  6950.0,
  2290.0,
  112.9,
  350.0,
  4000.0,
  7300.0,
  4370.0,
  3190.0,
  190.6,
  550.0,
  12160.0,
  252000.0,
  4360.0,
  7480.0,
  124300.0,
  4370.0,
  29.0,


In [47]:
# Channel indices which vary in time
mycruncher.variable()

[array([ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]),
 array([ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]),
 array([ 0,  1,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37])]

In [48]:
# Channel indices which are constant in time
mycruncher.constant()

[array([ 2,  3,  6, 26]), array([ 2,  3,  6, 26]), array([ 2,  3,  6, 26])]

In [49]:
# Sum of channel values over time
mycruncher.sums()

[array([ 4.80600000e+04,  7.58900000e+03,  0.00000000e+00,  0.00000000e+00,
         9.89530000e+05,  1.00830000e+04,  0.00000000e+00,  2.55375000e+04,
         2.55375000e+04,  2.55375000e+04,  1.44128285e+05,  5.74874039e+03,
         5.57390046e+05, -4.14189357e-01,  2.00572446e+02,  1.55066591e+03,
        -3.54332400e+01,  1.52463849e+03,  9.82605120e+01,  1.48320427e+03,
        -2.38651510e+02, -1.95322500e+01, -1.30349040e+00, -4.35471540e+01,
         1.17890310e+02, -1.10558919e+01,  0.00000000e+00,  2.62518510e+05,
         1.95963330e+04,  1.82839544e+05,  8.71292238e+05,  4.83505880e+05,
        -5.12910800e+04,  4.01501454e+03,  1.58754217e+02,  3.98490080e+02,
         2.08073147e+01,  4.32214800e+05]),
 array([ 4.80600000e+04,  7.58870100e+03,  0.00000000e+00,  0.00000000e+00,
         1.02185900e+06,  1.03015900e+04,  0.00000000e+00,  2.55375000e+04,
         2.55375000e+04,  2.55375000e+04,  1.46322747e+05,  5.81471523e+03,
         5.64074042e+05, -4.29132560e-01,  2

In [50]:
# Sum of channel values over time to the second power
mycruncher.sums_squared()

[array([2.99066700e+06, 7.23318632e+04, 0.00000000e+00, 0.00000000e+00,
        2.45104090e+09, 2.53956520e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 3.26158758e+07, 5.51322865e+04,
        5.18293747e+08, 1.52310134e-02, 3.39176067e+02, 6.81489156e+03,
        2.66962590e+02, 6.77778710e+03, 3.74944460e+02, 6.92880700e+03,
        5.28852071e+02, 8.93030853e+02, 2.49956593e+00, 5.45906269e+01,
        1.66439403e+02, 4.62125266e-01, 0.00000000e+00, 1.84694760e+08,
        4.01064942e+08, 5.08073920e+08, 2.49756762e+09, 2.46152297e+09,
        1.85959911e+09, 2.71241855e+04, 7.06899358e+01, 4.08180563e+02,
        1.37977617e+00, 3.49188164e+09]),
 array([2.99066700e+06, 7.23262441e+04, 0.00000000e+00, 0.00000000e+00,
        2.61382196e+09, 2.65084568e+05, 0.00000000e+00, 1.99462425e+06,
        1.99462425e+06, 1.99462425e+06, 3.34249703e+07, 5.64178526e+04,
        5.30965262e+08, 1.81228244e-02, 4.09763475e+02, 7.04301256e+03,
        2.74098853e+02

In [51]:
# Sum of channel values over time to the third power
mycruncher.sums_cubed()

[array([ 1.92288060e+08,  6.94298326e+05,  0.00000000e+00,  0.00000000e+00,
         6.09512315e+12,  6.40731260e+06,  0.00000000e+00,  1.65847398e+08,
         1.65847398e+08,  1.65847398e+08,  8.23177046e+09,  5.53178222e+05,
         5.04221025e+11, -1.55543032e-04,  3.65344809e+02,  2.58163535e+04,
         1.20947939e+01,  2.46568356e+04,  2.05625057e+02,  2.45975817e+04,
         7.10796755e+01,  9.96182890e+01, -1.48520740e-02, -4.35589968e+00,
         1.53278176e+01, -1.34159296e-02,  0.00000000e+00,  1.01753538e+11,
        -3.22542353e+10,  1.49847668e+11,  5.84157948e+12,  1.43604884e+12,
         5.65690138e+11,  1.93195077e+05,  2.86981674e+01,  3.38936129e+02,
         7.67175294e-02,  3.45922125e+12]),
 array([ 1.92288060e+08,  6.94219888e+05,  0.00000000e+00,  0.00000000e+00,
         6.71230136e+12,  6.83293668e+06,  0.00000000e+00,  1.65847398e+08,
         1.65847398e+08,  1.65847398e+08,  8.47749243e+09,  5.72455267e+05,
         5.22686792e+11, -2.15949030e-04,  4

In [52]:
# Sum of channel values over time to the fourth power
mycruncher.sums_fourth()

[array([1.27193675e+10, 6.72058716e+06, 0.00000000e+00, 0.00000000e+00,
        1.52191817e+16, 1.61942556e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.23468362e+12, 5.63005878e+06,
        4.97577410e+14, 2.75297219e-06, 4.38130767e+02, 1.06767943e+05,
        1.74151600e+02, 1.01872083e+05, 4.17986394e+02, 1.04873960e+05,
        9.00199729e+02, 2.22800798e+03, 2.07158246e-02, 7.26447975e+00,
        6.04853833e+01, 5.39994955e-04, 0.00000000e+00, 6.62801332e+13,
        3.18222414e+14, 4.30309506e+14, 1.45026648e+16, 1.32816255e+16,
        1.00332283e+16, 1.40368241e+06, 1.20486157e+01, 3.26802571e+02,
        4.46744190e-03, 3.85369085e+16]),
 array([1.27193675e+10, 6.71962967e+06, 0.00000000e+00, 0.00000000e+00,
        1.73077680e+16, 1.76438826e+08, 0.00000000e+00, 1.41881502e+10,
        1.41881502e+10, 1.41881502e+10, 2.30428942e+12, 5.89126197e+06,
        5.21876656e+14, 4.09519763e-06, 6.40737567e+02, 1.13515183e+05,
        1.71822081e+02

In [53]:
# Second moment of the timeseries for each channel
mycruncher.second_moments()

[array([1.33666667e+02, 5.37563724e-01, 0.00000000e+00, 0.00000000e+00,
        1.53384134e+06, 1.58591217e+02, 0.00000000e+00, 1.47370425e+03,
        1.47370425e+03, 1.47370425e+03, 8.34218479e+03, 1.73206485e+01,
        1.62826473e+05, 1.87476156e-05, 3.60739343e-01, 4.76022226e+00,
        3.31329783e-01, 4.83865350e+00, 4.53046962e-01, 5.22143747e+00,
        5.71470377e-01, 1.11430033e+00, 3.11790852e-03, 6.51974319e-02,
        1.86127885e-01, 3.86423328e-04, 0.00000000e+00, 1.23167724e+05,
        5.00106768e+05, 5.82195026e+05, 1.93484970e+06, 2.70869585e+06,
        2.31749656e+06, 8.73772149e+00, 4.89708356e-02, 2.62091685e-01,
        1.04777949e-03, 4.06824110e+06]),
 array([1.33666667e+02, 5.37621765e-01, 0.00000000e+00, 0.00000000e+00,
        1.63571375e+06, 1.65538992e+02, 0.00000000e+00, 1.47370425e+03,
        1.47370425e+03, 1.47370425e+03, 8.35886306e+03, 1.77365476e+01,
        1.66963034e+05, 2.23382251e-05, 4.28063595e-01, 4.87968906e+00,
        3.41619997e-01

In [54]:
# Third moment of the timeseries for each channel
mycruncher.third_moments()

[array([ 0.00000000e+00,  1.04578434e+00,  0.00000000e+00,  0.00000000e+00,
         3.94767725e+07,  1.54227646e+01,  0.00000000e+00,  3.36895113e+04,
         3.36895113e+04,  3.36895113e+04, -5.20236650e+04, -5.19943844e+01,
        -4.73889823e+07, -1.64965164e-07,  1.69420113e-01, -2.67129717e+00,
         5.91565473e-02, -3.74347926e+00,  8.81354619e-02, -4.64587100e+00,
         6.25981817e-01,  2.05897979e-01, -3.31601828e-06,  5.35617821e-03,
        -6.62345831e-02,  1.88154176e-06,  0.00000000e+00, -2.92704786e+07,
        -7.69871917e+07, -2.23500517e+08, -3.08118295e+08, -3.33225423e+09,
         1.15168683e+09, -1.61412350e+01, -1.07475623e-03, -9.11508028e-02,
        -3.40501759e-06, -2.42407639e+09]),
 array([ 0.00000000e+00,  1.04733005e+00,  0.00000000e+00,  0.00000000e+00,
         4.34894071e+07,  1.63269772e+01,  0.00000000e+00,  3.36895113e+04,
         3.36895113e+04,  3.36895113e+04, -9.31343873e+04, -5.41397262e+01,
        -4.94188476e+07, -2.33542671e-07,  2

In [55]:
# Fourth moment of the timeseries for each channel
mycruncher.fourth_moments()

[array([3.21601332e+04, 3.44360404e+00, 0.00000000e+00, 0.00000000e+00,
        2.43097390e+12, 2.55095341e+04, 0.00000000e+00, 3.39569151e+06,
        3.39569151e+06, 3.39569151e+06, 1.58498572e+08, 5.15307667e+02,
        4.55459785e+10, 3.06556334e-09, 2.37642092e-01, 3.28922739e+01,
        2.23991186e-01, 3.73738647e+01, 4.37451084e-01, 4.61649792e+01,
        1.55761531e+00, 2.79764035e+00, 2.57913196e-05, 9.06909402e-03,
        8.98454452e-02, 3.00027025e-07, 0.00000000e+00, 3.02030150e+10,
        4.03019005e+11, 5.56559180e+11, 4.31031003e+12, 1.85725589e+13,
        1.27638339e+13, 1.27548618e+02, 2.80917505e-03, 1.38923674e-01,
        1.23362554e-06, 4.61511943e+13]),
 array([3.21601332e+04, 3.45256143e+00, 0.00000000e+00, 0.00000000e+00,
        2.76447638e+12, 2.77911904e+04, 0.00000000e+00, 3.39569151e+06,
        3.39569151e+06, 3.39569151e+06, 1.57628652e+08, 5.41849680e+02,
        4.80084271e+10, 4.57357607e-09, 3.25917050e-01, 3.38973659e+01,
        2.17601724e-01

In [56]:
# Mean of channel values over time
mycruncher.means()

[[60.0,
  9.474406991260869,
  0.0,
  0.0,
  0.0076519825218477056,
  0.0033865418227215756,
  0.0,
  0.002866609238451929,
  0.01073330711610486,
  0.0,
  -0.0011934993757802812,
  1235.3682896379526,
  12.588014981273426,
  0.0,
  31.882022471910112,
  31.882022471910112,
  31.882022471910112,
  179.93543645443197,
  7.176954299625444,
  695.867722596754,
  -0.0005170903333333338,
  0.25040255430711617,
  1.9359125018726577,
  -0.044236254681648,
  0.0,
  1.9034188451935086,
  0.1226722996254682,
  0.0,
  1.8516907228464454,
  -0.2979419600499377,
  0.0,
  47.88283395755306,
  55.30990012484407,
  56.599937578027685,
  -0.0243848314606742,
  -0.001627328838951311,
  -0.05436598501872664,
  0.14717891385767798,
  -0.01380261154806491,
  0.0,
  32.518601747815296,
  -0.049951031423220875,
  -0.13459880898876392,
  0.04102214631710354,
  0.4493361922596746,
  -0.24992296504369546,
  -0.06272093196004998,
  -4.7685468164793874e-05,
  0.003685510611735314,
  106.16842821473152,
  -9.16578

In [57]:
# Median of channel values over time
mycruncher.medians()

[[60.0,
  9.4,
  0.0,
  0.0,
  -0.0152,
  -0.0276,
  0.0,
  -0.0129,
  -0.00186,
  0.0,
  0.0605,
  2300.0,
  24.0,
  0.0,
  0.0,
  0.0,
  0.0,
  192.0,
  10.0,
  974.0,
  9.87e-06,
  -0.0882,
  3.34,
  -0.0895,
  0.0,
  3.4,
  0.0527,
  0.0,
  3.41,
  -0.479,
  0.0,
  53.8,
  63.8,
  63.4,
  -0.076,
  0.00191,
  -0.0533,
  0.278,
  -0.0154,
  0.0,
  34.1,
  -0.0382,
  -0.0987,
  0.0517,
  1.07,
  0.0909,
  -0.049,
  -2.78e-05,
  0.0108,
  157.0,
  11.2,
  281.0,
  -162.0,
  5850.0,
  -2.82,
  167.0,
  60.5,
  343.0,
  -1170.0,
  6220.0,
  -0.873,
  170.0,
  -85.7,
  300.0,
  1780.0,
  6190.0,
  3.76,
  -6.56,
  1620.0,
  -2.4,
  38.2,
  1650.0,
  -2.35,
  -12.2,
  1650.0,
  -2.56,
  463.0,
  145.0,
  564.0,
  2320.0,
  1160.0,
  -312.0,
  472.0,
  -7.19,
  -3410.0,
  2340.0,
  121.0,
  0.842,
  572.0,
  -5.34,
  -6820.0,
  1720.0,
  45900.0,
  -0.389,
  1460.0,
  23300.0,
  1.31,
  205.0,
  82.7,
  0.0,
  0.0,
  238.0,
  75.5,
  0.0,
  0.0,
  344.0,
  61.1,
  0.0,
  0.0,
  547.0,
  47

In [58]:
# Standard deviation of channel values over time
mycruncher.stddevs()

[[11.561430130683084,
  0.7331873733878951,
  0.0,
  0.0,
  0.4221677852002905,
  0.2988545890176859,
  0.0,
  0.29680050948971887,
  0.2804489345312724,
  0.0,
  0.28325223310950853,
  1238.4834854625656,
  12.593300487097336,
  0.0,
  38.38885575394094,
  38.38885575394094,
  38.38885575394094,
  91.3355614638876,
  4.161808318042927,
  403.51762400937525,
  0.00432985168909129,
  0.6006158033858908,
  2.181793358239024,
  0.5756125282671347,
  0.0,
  2.1996939568646208,
  0.6730876332012791,
  0.0,
  2.285046491763002,
  0.7559565975672955,
  0.0,
  17.66320923227619,
  15.782304428494266,
  14.566749967488587,
  1.0556042475104248,
  0.05583823531576016,
  0.2553378779299062,
  0.4314254102373269,
  0.01965765316500327,
  0.0,
  5.005030282853597,
  0.05034601556996795,
  0.3756090844194291,
  0.07129516531590052,
  2.709583970822169,
  0.6013534715670306,
  0.15137751063909416,
  0.004025696466459002,
  0.1558485086064321,
  104.68076582462815,
  117.13564128733154,
  170.98398133

In [59]:
# Skew of channel values over time
mycruncher.skews()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:363: RuntimeWarning: invalid value encountered in divide
  return self.third_moments / np.sqrt(self.second_moments) ** 3


[array([ 0.        ,  2.65336626,         nan,         nan,  0.02078122,
         0.00772224,         nan,  0.59549649,  0.59549649,  0.59549649,
        -0.06827807, -0.72129051, -0.7212567 , -2.03223151,  0.78194229,
        -0.25720616,  0.3101788 , -0.35171318,  0.28902529, -0.38938752,
         1.44900917,  0.17504459, -0.01904679,  0.32174313, -0.82483607,
         0.24769592,         nan, -0.67714933, -0.21768293, -0.50312488,
        -0.11448448, -0.74747688,  0.32644179, -0.62494164, -0.09917534,
        -0.67932975, -0.10039556, -0.29541754]),
 array([ 0.        ,  2.65685773,         nan,         nan,  0.02078847,
         0.00766576,         nan,  0.59549649,  0.59549649,  0.59549649,
        -0.12186788, -0.72479038, -0.72437278, -2.21204139,  0.78021022,
        -0.25864842,  0.22294603, -0.37447278,  0.44241491, -0.41560236,
         1.54672098,  0.17733024, -0.10541872,  0.23070655, -0.90416767,
         0.29340951,         nan, -0.7905145 , -0.26053364, -0.5054796 ,
  

In [60]:
# Kurtosis of channel values over time
mycruncher.kurtosis()

/Users/gbarter/devel/pCrunch/pCrunch/aeroelastic_output.py:367: RuntimeWarning: invalid value encountered in divide
  return self.fourth_moments / self.second_moments ** 2


[array([ 1.79999626, 11.91662575,         nan,         nan,  1.03328332,
         1.01424824,         nan,  1.56353482,  1.56353482,  1.56353482,
         2.27753858,  1.71766437,  1.71790841,  8.72204274,  1.82614957,
         1.45157486,  2.04037492,  1.59631635,  2.13129248,  1.69329444,
         4.76949898,  2.25313579,  2.65305849,  2.13355049,  2.59342195,
         2.00924873,         nan,  1.99093218,  1.61138777,  1.6420046 ,
         1.15136769,  2.53134453,  2.37652806,  1.67062648,  1.1713961 ,
         2.02241265,  1.12368238,  2.78849312]),
 array([ 1.79999626, 11.94504332,         nan,         nan,  1.03323302,
         1.01416014,         nan,  1.56353482,  1.56353482,  1.56353482,
         2.25600854,  1.72242627,  1.72217318,  9.16555105,  1.77864998,
         1.42357926,  1.86455742,  1.64399072,  2.34040176,  1.70666827,
         5.06379094,  2.73034727,  2.71665559,  2.09867975,  2.79245263,
         2.43625957,         nan,  2.08109812,  1.60028736,  1.67032893,
  

In [61]:
# Integration of channel values over time
mycruncher.integrated()

[[2400.0,
  378.97999999999786,
  0.0,
  0.0,
  0.3185869000000005,
  0.14550599999999986,
  0.0,
  0.1238327000000003,
  0.4336164500000013,
  0.0,
  -0.05739965000000015,
  49418.5000000001,
  503.5474999999994,
  0.0,
  1274.6250000000027,
  1274.6250000000027,
  1274.6250000000027,
  7193.41423,
  287.1878172,
  27845.280289999842,
  -0.020711242850000006,
  10.003839800000002,
  77.43830802500003,
  -1.7891495000000004,
  0.0,
  76.13219975000014,
  4.92485059999999,
  0.0,
  74.07748844999993,
  -11.894325499999992,
  0.0,
  1915.0549999999994,
  2211.9290000000015,
  2265.0244999999986,
  -1.0179124999999978,
  -0.0663082699999994,
  -2.1841702000000014,
  5.90296549999999,
  -0.5518263425000008,
  0.0,
  1301.1575000000018,
  -1.998296308500001,
  -5.37945729999999,
  1.6409869600000027,
  18.096164499999983,
  -9.984624749999996,
  -2.503213324999998,
  -0.0017465529999999578,
  0.14546470000000017,
  4247.790550000006,
  -373.28950000000145,
  12845.775000000003,
  11638.8000

In [62]:
# Special instance of the integration that specifically uses
# the Power channel string to integrate over time and calculate energy
mycruncher.compute_energy('GenPwr')

[49418.5000000001, 51033.00000000001, 50520.17499999997]